In [1]:
import torch
import numpy as np

In [2]:
# path2pretrained = '/gpfs/gpfs0/y.gusak/pretrained/torchvision/models_torch_0-4-0'
path2pretrained = "/gpfs/gpfs0/y.gusak/pretrained/DCP/imagenet/models_torch_1-0-1"

model_name = 'resnet18_imagenet'
# depth = 18
# pruning_rate = 0.3
# model_name = 'resnet{}_pruned{}'.format(depth, pruning_rate)

import sys
sys.path.append('/trinity/home/y.gusak/ReducedOrderNet/src')
from models.dcp.pruned_resnet import PrunedResNet

path2pretrained_model = '{}/{}.pth'.format(path2pretrained, model_name)

In [3]:
model = torch.load(path2pretrained_model)

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/gpfs0/y.gusak/pretrained/DCP/imagenet/models_torch_1-0-1/resnet18_imagenet.pth'

In [6]:
model

PrunedResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): PrunedBottleneck(
      (conv1): Conv2d(64, 45, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(45, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1

In [23]:
import sys
sys.path.append('../')
from model_utils import get_layer_names

layer_names, conv_layer_mask = get_layer_names(model)


fc_layer_mask = (1 - conv_layer_mask).astype(bool)

print(layer_names[conv_layer_mask][::-1])
print(layer_names[fc_layer_mask])

['layer4.1.conv2' 'layer4.1.conv1' 'layer4.0.downsample.0'
 'layer4.0.conv2' 'layer4.0.conv1' 'layer3.1.conv2' 'layer3.1.conv1'
 'layer3.0.downsample.0' 'layer3.0.conv2' 'layer3.0.conv1'
 'layer2.1.conv2' 'layer2.1.conv1' 'layer2.0.downsample.0'
 'layer2.0.conv2' 'layer2.0.conv1' 'layer1.1.conv2' 'layer1.1.conv1'
 'layer1.0.conv2' 'layer1.0.conv1' 'conv1']
['fc']


In [25]:
# if model_name == 'resnet18_imagenet':
if 'resnet18' in model_name:
    filtered_layer_names = list(filter(lambda l: 'downsample' not in l\
                                       and 'layer' in l,\
                                       layer_names[conv_layer_mask][::-1]
                                      ))
    filtered_layer_names = np.array(filtered_layer_names)
    
    gr_schedules = [
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 4)],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 8)],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 16)]]
    
# elif model_name == 'resnet50_imagenet':
elif 'resnet50' in model_name:
    filtered_layer_names = list(filter(lambda l: 'downsample' not in l\
                                   and 'layer' in l\
                                   and 'conv2' in l,\
                                   layer_names[conv_layer_mask][::-1]
                                  ))
    filtered_layer_names = np.array(filtered_layer_names)
    
    gr_schedules = [
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 4)],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 8)],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, [2, 8, 12])],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 16)]
    ]

In [26]:
gr_schedules[-1]

['layer4.1.conv2',
 'layer4.1.conv1',
 'layer4.0.conv2',
 'layer4.0.conv1',
 'layer3.1.conv2',
 'layer3.1.conv1',
 'layer3.0.conv2',
 'layer3.0.conv1',
 'layer2.1.conv2',
 'layer2.1.conv1',
 'layer2.0.conv2',
 'layer2.0.conv1',
 'layer1.1.conv2',
 'layer1.1.conv1',
 'layer1.0.conv2',
 'layer1.0.conv1']

In [8]:
np.split(filtered_layer_names, 8)

[array(['layer4.1.conv2', 'layer4.1.conv1'], dtype='<U14'),
 array(['layer4.0.conv2', 'layer4.0.conv1'], dtype='<U14'),
 array(['layer3.1.conv2', 'layer3.1.conv1'], dtype='<U14'),
 array(['layer3.0.conv2', 'layer3.0.conv1'], dtype='<U14'),
 array(['layer2.1.conv2', 'layer2.1.conv1'], dtype='<U14'),
 array(['layer2.0.conv2', 'layer2.0.conv1'], dtype='<U14'),
 array(['layer1.1.conv2', 'layer1.1.conv1'], dtype='<U14'),
 array(['layer1.0.conv2', 'layer1.0.conv1'], dtype='<U14')]

In [27]:
path2schedules = '/gpfs/gpfs0/y.gusak/musco_models/{}'.format(model_name)

for i, gr_schedule in enumerate(gr_schedules):
    if i == len(gr_schedules)-1:
        i=1000
    with open('{}/grschedule{}.txt'.format(path2schedules, i), 'w') as f:
        for item in gr_schedule:
            f.write("%s\n" % item)

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/gpfs0/y.gusak/musco_models/resnet18_pruned0.3/grschedule0.txt'